# Face Clustering for Photo Albums


## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.core.display import display
from ipywidgets import interact, interact_manual
from ipywidgets import Button, HBox, VBox, GridspecLayout, Layout
import cv2 
from matplotlib import pyplot as plt
import src.gui_controller as gui_controller




Using TensorFlow backend.


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999; // prevents collapsing of the image preview

<IPython.core.display.Javascript object>

##  Select File Path

In [3]:
image_folder = r"C:\AX";

## Extract Faces from Album / Image(s) and Extract Features

In [4]:
radio = widgets.RadioButtons(
    options=['Scanned Album', 'Image(s)'],
    description='Mode:',
    disabled=False
)
information_label = widgets.Label(value="If you have already extracted the faces once for this folder you can press 'Extract Features' right away")
button_extract_faces = Button(description="Extract Faces", button_style= 'info')
button_extract_embeddings = Button(description="Extract Features", button_style = 'primary')


def set_buttons_enabled(enabled):
    button_extract_faces.disabled = not enabled
    button_extract_embeddings.disabled = not enabled
    
def extract_faces(b):
    set_buttons_enabled(False)
    information_label.value = "Face Extraction is in progress. Please wait."
    gui_controller.extract_faces_from_folder(image_folder, isAlbum)
    information_label.value = "Faces have been extracted, you can start the feature extraction now"
    set_buttons_enabled(True)
    
def extract_embeddings(b):
    global embeddings
    global face_imgs
    set_buttons_enabled(False)
    information_label.value = "Feature Extraction is in progress. Please wait."
    isAlbum = False
    if radio.value == 'Scanned Album':
        isAlbum=True
    embeddings, face_imgs = gui_controller.get_embeddings_and_imgs(image_folder)
    information_label.value = "Features have been extracted, you can now start clustering"
    set_buttons_enabled(True) 
    
button_extract_faces.on_click(extract_faces)
button_extract_embeddings.on_click(extract_embeddings)
display(radio)
display(button_extract_faces)
display(button_extract_embeddings)
display(information_label)

RadioButtons(description='Mode:', options=('Scanned Album', 'Image(s)'), value='Scanned Album')

Button(button_style='info', description='Extract Faces', style=ButtonStyle())

Button(button_style='primary', description='Extract Features', style=ButtonStyle())

Label(value="If you have already extracted the faces once for this folder you can press 'Extract Features' rig…

## Cluster Faces

In [7]:
threshold_slider = widgets.IntSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    description='Threshold:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)
@interact
def cluster(threshold = threshold_slider):
    print("Class for every Face (-1 stands for undetected)\n")
    predictions, cluster_count, avg_imgs_per_cluster, closest_clusters = gui_controller.get_clusters(embeddings,threshold)
    print("No of Clusters: " +  "\x1b[31m "+str(cluster_count)+ "\x1b[0m")
    print("Avg Images per Cluster: " +  "\x1b[31m "+str(avg_imgs_per_cluster)+ "\x1b[0m" )    
    print("\n Clusters:")
    print(predictions)
    print(closest_clusters)
    
preview_button =widgets.Button(
    description='Preview anzeigen',
    disabled=False,
    button_style='info', 
    tooltip='Zeigt Preview für die generierten Cluster an',
    icon='image' # (FontAwesome names without the `fa-` prefix)
)


out = widgets.Output(
    max_height = 100,
    layout = {'border':'1px solid black'}

)
cluster_box = None

def show_preview(b):
    with out:
        out.clear_output()
        threshold = threshold_slider.value
        predictions, cluster_count, avg_imgs_per_cluster, closest_clusters = gui_controller.get_clusters(embeddings,threshold)
        min_cluster_size  = 2
        max_images_per_plot = 3
        labels = np.unique(predictions)
        labels = labels[labels > -1]
        checkboxes = []
        all_clusters_box = []
        for idx, label in enumerate(labels):
            image_indices = np.where(predictions == label)[0]
            closest_cluster_indices = np.where(predictions == closest_clusters[label])[0]
            if image_indices.shape[0] >= min_cluster_size:
                cluster_box = []
                grid = GridspecLayout(1, 7, layout = {'overflow':'hidden'})
                columns = min(image_indices.shape[0], max_images_per_plot)
                for i in range(columns):
                    file = open(face_imgs[image_indices[i]], "rb")
                    image = file.read()
                    img = widgets.Image(
                        value= image,
                        format='png',
                        width=200,
                        height=200,
                    )
                    grid[0,i] = img 

                cbox = widgets.Checkbox(False, description='Same?', layout = Layout(width = 'auto', justify_content = 'center'), indent = False)
                grid[0,3] = cbox
                columns = min(closest_cluster_indices.shape[0], max_images_per_plot)
                for i in range(columns):
                    file = open(face_imgs[closest_cluster_indices[i]], "rb")
                    image = file.read()
                    img = widgets.Image(
                        value= image,
                        format='png',
                        width=200,
                        height=200,
                    )
                    grid[0,4+i] = img 
                name = widgets.Text(
                    value=str(idx),
                    placeholder='Enter Name',
                    description='Name:',
                    disabled=False
                )
                cluster_box.append(grid)
                cluster_box.append(name)
                all_clusters_box.append(VBox(cluster_box))
        cluster_box = VBox(all_clusters_box)
        display(cluster_box)
preview_button.on_click(show_preview)      
display(preview_button)

display(out)


interactive(children=(IntSlider(value=30, continuous_update=False, description='Threshold:', min=1), Output())…

Button(button_style='info', description='Preview anzeigen', icon='image', style=ButtonStyle(), tooltip='Zeigt …

Output(layout=Layout(border='1px solid black'))